# This code helps you to parse data from НАШ.ДОМ.РФ web map

You can look at the map of new buildings [here](https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA-%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2/%D0%BA%D0%B0%D1%80%D1%82%D0%B0?bounds=-21.303352635369155%2C-60.46875099999995%2C84.10483966467523%2C179.99999900000003&objectIds=42470%2C41762%2C49513)

In [1]:
# Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)
print('Ядро перезапущено, можно продолжать работать!')

Ядро перезапущено, можно продолжать работать!


: 

### Import libraries


In [1]:
import pandas as pd
import geopandas as gpd

# For working with requests
import requests
import json

import folium
import matplotlib
import mapclassify

### Define bounding box (coordinates that limit search area)

In [2]:
latBounds = [56, 57]
longBounds = [60, 61]
bbbox = {'long': longBounds, 'lat': latBounds}

### URL API

In [3]:

base_url = "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api"
test_id = "40958"

url_get_points  = f"{base_url}/kn/object/map?offset=0&latitudePoint1=60&latitudePoin2=61&longitudePoint1=56&longitudePoint2=57"
url_get_point_info = f"{base_url}/object/{test_id}"

### Request list of available objects in a selected area

In [4]:
r = requests.get(url_get_points)
print(r.status_code)
data = r.json()

200


### Save objects IDs to an array

In [11]:
list_objects = data['data']['list']

# filtered_list_objects = [dict for dict in list_objects if dict['latitude'] ]
# objects_df = pd.DataFrame.from_dict(filtered_list_objects)
# objectsIds = [o['objId'] for o in filtered_list_objects if 'objId' in o]

objects_df = pd.DataFrame.from_dict(list_objects)

objectsIds = objects_df.loc[(  objects_df ['latitude'] <= 57 ) & ( objects_df ['latitude'] >= 56 )  & ( objects_df ['longitude'] >= 60 ) & ( objects_df ['longitude'] <= 61 )]['objId']
len(objectsIds)



851

### Function to GET properties for all objects

In [8]:
# def get_objects(bbox):
#     base_url = "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api"
#     url_get_points  = f"{base_url}/kn/object/map?offset=0&limit=999999&latitudePoint1={bbox['lat'][0]}&latitudePoin2={bbox['lat'][1]}&longitudePoint1={bbox['long'][0]}&longitudePoint2={bbox['long'][1]}"
#     r = requests.get(url_get_points)
#     listObjects = r.json()['data']['list']
#     objectsIds_all = [o['objId'] for o in listObjects if 'objId' in o]
#     objectsIds = objectsIds_all[0:100]
#     df_final = pd.DataFrame()

#     if r.status_code == 200:
#         for n, i in enumerate(objectsIds):
#             url_get_point_info = f"{base_url}/object/{i}"
#             r2 = requests.get(url_get_point_info)
#             if r2.status_code == 200:
#                 data = r2.json()['data']
#                 data_df = pd.DataFrame.from_dict([data])
#                 df_final = pd.concat([df_final, data_df], axis=0)
#             else:
#                 return r2.status_code
    
#     else:
#         return r.status_code

#     return  df_final


In [16]:
def get_objects(bbox):
    base_url = "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api"
    url_get_points  = f"{base_url}/kn/object/map?offset=0&latitudePoint1=60&latitudePoin2=61&longitudePoint1=56&longitudePoint2=57"
    r = requests.get(url_get_points)
    listObjects = r.json()['data']['list']
    objects_df = pd.DataFrame.from_dict(list_objects)
    objectsIds = objects_df.loc[( objects_df ['latitude'] <= bbox['lat'][1] ) & ( objects_df ['latitude'] >= bbox['lat'][0] ) & ( objects_df ['longitude'] >= bbox['long'][0] ) & ( objects_df ['longitude'] <= bbox['long'][1])]['objId']
    df_final = pd.DataFrame()

    if r.status_code == 200:
        for n, i in enumerate(objectsIds):
            url_get_point_info = f"{base_url}/object/{i}"
            r2 = requests.get(url_get_point_info)
            if r2.status_code == 200:
                data = r2.json()['data']
                data_df = pd.DataFrame.from_dict([data])
                df_final = pd.concat([df_final, data_df], axis=0)
            else:
                continue

    
    else:
        return r.status_code

    return  df_final

In [17]:
objects = get_objects(bbbox)

In [18]:
objects

### Exporting data to a CSV file

In [46]:
objects.to_csv('./objects_ekat.csv', index=False, index_label=False)

### Creating Spatial Data Frame from a Data Frame

In [ ]:
spatial = gpd.GeoDataFrame(objects, geometry=gpd.points_from_xy(objects.objLkLongitude, objects.objLkLatitude))

### Explore the data 

In [ ]:
spatial.explore()